<html>

<head>
<meta charset="UTF-8">
<title>Title of the document</title>
</head>

<body>
<h1 id="title">Plots from X-ray absorption calculations using ORCA</h1>
<p>Presentation of the feature engineering applied for ORCA outputs</p>
<p>Data presented here, as new inputs for this jupyter-notebook, comes from the written-shell pipeline created to extract X-ray excited-state features from some specific pair of atoms group (e.g. pair amino acid).</p>
<p>Outputs here are filtered features that account for specific transitions (coreMO -> virtualMO) for the specific pair of atoms group.</p>
</body>
</html>

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import csv as csv
from matplotlib.pyplot import figure
from __future__ import print_function
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

____________________________

In [15]:
from scipy.optimize import leastsq
from scipy.optimize import curve_fit
from scipy.signal import find_peaks

In [16]:
plt.rcParams['axes.labelsize'] = 24
plt.rcParams['axes.titlesize'] = 24

--------------

<html>

<head>
<meta charset="UTF-8">
<title>Title of the document</title>
</head>

<body>
<h1 id="title">Löwdin Population analysis from </h1>
<h2 id="title"> X-ray absorption spectra calculated using PNO-DFT/ROCIS zora </h2>
<h3 id="title">    System of study: F ${---}$Y</h3>
<h4 id="title">   non-covalent interaction distances varying from 2,5 to 11,0 $\dot{A}$ </h4>
</body>
</html>


#### Loading matrices (data from the pipeline's outputs)
Data in .csv format
##### residue called as A is acting as one at the ground state amino acid
> ##### resA core MO
##### residue called as B is acting as one at the virtual state amino acid
> ##### resB virtual MO

In a pair-aa interaction, e.g. Phe-Tyr, Phenylalanine stands for resA core MO and Tyrosine for resB virtual MO. And after, Tyrosine stands for resA core MO and Phenylalanine for resB virtual MO. Both are calculated and are within the spectra.

> ##### resA core MO - Phenylalanine

In [25]:
#Uncomment the original range to see more peaks outside the expected range
resa_phe="resA_MOcore_FY_"
post=".csv"
#list_suf = [25, 26, 27, 28, 29, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110]
list_suf = [27, 35, 55, 75, 90, 110]
for ii in list_suf:
    globals()['resa_mocore_%s_FY_raw' % str(ii)] = pd.read_csv(resa_phe+str(ii)+post, delimiter=',', index_col='num-1')

resa_mocore_27_FY_raw.head()

,sym,lvl,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
num-1,,,,,,,,,,,,,,,,,,,,
1,C,s,0.0,0.0,0.0,0.0,99.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,C,s,0.0,0.0,0.0,0.0,0.0,0.0,0.3,0.3,0.0,0.0,0.0,0.0,0.0,3.6,95.4,0.0,0.0,0.0
11,C,s,0.0,0.0,0.0,0.0,0.0,0.0,0.3,98.7,0.0,0.0,0.0,0.0,0.0,0.2,0.5,0.0,0.0,0.0
2,C,s,99.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,C,s,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,99.3,0.0,0.0,0.0,0.0,0.0,0.0


> ##### resB virt MO - Tyrosine

In [26]:
resb_tyr="resB_MOcore_FY_"
post=".csv"
for ii in list_suf:
    globals()['resb_movirt_%s_FY_raw' % str(ii)] = pd.read_csv(resb_tyr+str(ii)+post, delimiter=',', index_col='num-1')

resb_movirt_27_FY_raw.head()

,sym,lvl,92,93,94,95,96,97,98,99,100,101,102,103
num-1,,,,,,,,,,,,,,
23,atom,lvlMO,0.0,0.0,0.0,0.0,0.9,2.9,0.6,1.2,3.2,0.6,1.3,1.2
24,atom,lvlMO,0.4,0.8,0.0,0.0,2.2,3.8,1.5,1.7,3.8,1.1,0.5,1.9
25,atom,lvlMO,0.2,0.0,0.0,0.0,1.2,22.2,7.7,12.8,1.3,2.1,3.0,3.5
26,atom,lvlMO,0.0,0.0,0.0,0.0,0.3,10.1,3.7,5.8,0.5,0.8,1.8,1.4
27,atom,lvlMO,0.0,0.0,0.0,0.0,0.4,7.5,2.7,7.7,0.5,0.5,0.8,0.8


<html>

<head>
<meta charset="UTF-8">
<title>Title of the document</title>
</head>

<body>
<h1 id="title">Features from  X-ray excited-state properties: </h1>
<h2 id="title">1. The number of transitions </h2>
<h2 id="title">2. The transitions averaged probability </h2>
<h2 id="title">3. The oscillator strength using electronic dipole moment as operator </h2>
<h2 id="title">4. The oscillator strength using velocity dipole moment as operator </h2>
</body>
</html>

#### coreMO vs. virtual MO (core/virt MO)
#### resA(Phe) --> resB(Tyr)

> ### 1. core/virt MO as function of the number of transitions

In [27]:
resab_fy="corevirtMO_matrix_FY_"
post=".csv"
for ii in list_suf:
    globals()['corevirtMO_matrix_%s_FY_raw' % str(ii)] = pd.read_csv(resab_fy+str(ii)+post, delimiter=',', index_col='virt\core')

corevirtMO_matrix_27_FY_raw.head()

,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
virt\core,,,,,,,,,,,,,,,,,,
92,3,2,5,1,1,1,4,3,1,4,2,1,4,2,5,4,3,4
93,5,2,5,1,2,1,5,4,1,3,4,1,4,4,4,4,4,2
94,5,3,5,1,2,1,6,5,2,5,4,2,4,4,4,4,3,3
95,5,4,5,1,1,1,5,5,2,5,4,3,4,4,4,6,4,2
96,5,3,4,1,1,1,4,4,1,3,4,2,3,4,5,3,3,3


> ### 2. core/virt MO as function of the number of transitions averaged probability

In [29]:
resab_ts_fy="corevirtMO_matrix_tspb_FY_"
post=".csv"
for ii in list_suf:
    globals()['corevirtMO_matrix_tspb_%s_FY_raw' % str(ii)] = pd.read_csv(resab_ts_fy+str(ii)+post, delimiter=',', index_col='virt\core')

corevirtMO_matrix_tspb_27_FY_raw.head()

,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
virt\core,,,,,,,,,,,,,,,,,,
92,0.326367,0.995100,0.029640,1.0000,0.99990,0.9861,0.01815,0.295800,0.98730,0.09605,0.489550,0.9902,0.037050,0.490450,0.050960,0.025875,0.326467,0.220275
93,0.158700,0.995150,0.019960,0.9998,0.49995,0.9792,0.07248,0.226000,0.99150,0.27800,0.198375,0.9904,0.024950,0.194825,0.021450,0.012825,0.237575,0.497650
94,0.187600,0.329267,0.020520,0.9996,0.49725,0.9779,0.00755,0.009100,0.49715,0.00522,0.234500,0.4939,0.025650,0.213050,0.055175,0.021775,0.325967,0.328333
95,0.182540,0.013950,0.012640,0.9999,0.99450,0.9810,0.01278,0.148340,0.49465,0.04972,0.228175,0.0186,0.015800,0.201050,0.061350,0.006717,0.239350,0.499000
96,0.145560,0.329800,0.041725,1.0000,0.99990,0.9893,0.23300,0.231575,0.98660,0.32720,0.181950,0.4947,0.055633,0.163550,0.010700,0.124800,0.322467,0.327733


> ### 3. core/virt MO as function of the oscillator strength using electronic dipole moment as operator

In [30]:
resab_fosce_fy="corevirt_fosce_FY_"
post=".csv"
for ii in list_suf:
    globals()['corevirt_mo_fosce_%s_FY_raw' % str(ii)] = pd.read_csv(resab_fosce_fy+str(ii)+post, delimiter=',', index_col='virt\core')

corevirt_mo_fosce_27_FY_raw.head()

,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
virt\core,,,,,,,,,,,,,,,,,,
92,0.006312,0.000083,4.705410e-06,1.935600e-05,0.000021,0.128511,0.000143,0.000105,0.000032,3.104510e-06,0.009469,0.000162,5.881770e-06,0.007402,0.000371,2.881990e-06,0.000122,0.000039
93,0.010611,0.000013,4.595310e-07,2.028590e-06,0.000034,0.117189,0.000241,0.000183,0.000003,2.301930e-06,0.013264,0.000025,5.744140e-07,0.009407,0.000948,3.784730e-07,0.000012,0.000006
94,0.000637,0.010160,1.431480e-06,1.415330e-05,0.000092,0.140934,0.000010,0.000006,0.000010,3.675190e-07,0.000797,0.015241,1.789350e-06,0.001044,0.000105,1.099700e-06,0.000045,0.000009
95,0.007795,0.000463,6.364410e-08,6.009400e-07,0.000016,0.004856,0.000391,0.000065,0.000007,7.790250e-05,0.009744,0.000618,7.955510e-08,0.004950,0.000826,2.381120e-07,0.000003,0.000002
96,0.029981,0.000148,1.351800e-06,1.540000e-06,0.000002,0.000745,0.002580,0.000722,0.000003,9.736350e-06,0.037477,0.000221,1.802410e-06,0.037579,0.002152,4.653950e-06,0.000009,0.000025


> ### 4. core/virt MO as function of the oscillator strength using velocity dipole moment as operator

In [38]:
resab_foscv_fy="corevirt_foscv_FY_"
post=".csv"
for ii in list_suf:
    globals()['corevirt_mo_foscv_%s_FY_raw' % str(ii)] = pd.read_csv(resab_foscv_fy+str(ii)+post, delimiter=',', index_col='virt\core')

corevirt_mo_foscv_27_FY_raw.head()

,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
virt\core,,,,,,,,,,,,,,,,,,
92,0.005975,0.000041,4.783430e-06,3.075500e-05,0.000017,0.120655,0.000135,0.000099,0.000083,5.004890e-06,0.008962,0.000076,5.979290e-06,0.007028,0.000352,3.355180e-06,0.000126,0.000037
93,0.010412,0.000003,5.446780e-07,2.630470e-06,0.000032,0.109963,0.000235,0.000165,0.000004,2.592780e-06,0.013015,0.000005,6.808480e-07,0.008935,0.000900,4.776060e-07,0.000015,0.000007
94,0.000624,0.008359,2.161160e-06,1.693420e-05,0.000076,0.132422,0.000010,0.000005,0.000018,5.512080e-07,0.000780,0.012539,2.701440e-06,0.001009,0.000102,1.577630e-06,0.000066,0.000015
95,0.008102,0.000414,4.348590e-08,7.849220e-07,0.000011,0.004755,0.000416,0.000067,0.000009,7.317190e-05,0.010127,0.000552,5.435740e-08,0.004884,0.000830,2.398790e-07,0.000002,0.000002
96,0.030624,0.000139,1.786400e-06,2.052000e-06,0.000003,0.000845,0.002267,0.000535,0.000004,1.264180e-05,0.038281,0.000209,2.381860e-06,0.038319,0.002192,6.787840e-06,0.000011,0.000026


### Main functions to present data according to the features

In [32]:
def selecting_atm_matrix(df, atoms_list):
    """
    it returns MO matrix (df) having just the atoms in atoms_list.
    """
    col_list = df.index.tolist() #listing index (elements from column 0)
    #dff = df.loc[:, (df != 0).any(axis=0)] #removing zero columns
    return df.loc[[i for i in atoms_list if i in col_list]] #returning just columns in the both lists intersection
#atoms_core=resa_mocore_GFG9Y_raw.index.tolist() #all C-core atoms
#[i for i in atoms_a if i in atoms_core] #atoms_a intersection atoms_core

In [33]:
def nonzero_mo_matrix(df):
    """
    it returns MO matrix (df) having just non-zero MO population.
    """
    return df.loc[:, (df != 0).any(axis=0)] #removing zero columns

In [34]:
def crop_by_loewdin_p(df,pop):
    """
    it returns MO matrix (df) having just a Loewdin MO population contribution greater than (100*pop)%.
    pop is a number between 0 and 1.
    """
    cols = [col for col, val in df.sum()[2:].iteritems() if val > (pop*100)]
    #iteritem() create a zip with the index_col and the values
    cols.insert(0,'lvl')
    cols.insert(0,'sym')
    #to add on the top the first two columns removed in the previous sum() evaluation
    return df[cols] #showing the fisrt two columns and those having population greater than pop*100%

In [53]:
def cropping_matrix(df, df1, df2):
    """
    it returns a cropped MO matrix from df using as parameters the indices in df1 and df2.
    df1 index are column names from df and df2 index are the rows from df
    """
    try:
        dff = df[df1.axes[1][2:].tolist()] #saving specific columns
    except KeyError:
        tmp_mo = [item for item in df1.axes[1][2:].tolist() if item in df.axes[1][2:].tolist()]
        #removing items in the first list that are not in the second one
        dff = df[tmp_mo]
    try:
        return dff.loc[[int(i) for i in df2.axes[1][2:].tolist()]] #returning specific rows
    except KeyError:
        tmp_mo2 = [item for item in df2.axes[1][2:].tolist() if item in dff.T.axes[1][2:].tolist()]
        # dff.T to obtain the index_col as the head row
        return dff.loc[[int(i) for i in tmp_mo2]] #returning specific rows
# the first two elements in df2.axes[1] and df1.axes[1] are "sym" and "lvl", that's why I used df.axes[1][2:]

<!DOCTYPE html>
<html>

<head>
<meta charset="UTF-8">
<title>Title of the document</title>
</head>

<body>
<h3 id="title">Reducing Löwdin Population MO matrices to resA boundaries</h3>
</body>
</html>

In [37]:
for ii in list_suf:
    globals()['resa_mocore_%s_FY' % str(ii)] = nonzero_mo_matrix(globals()['resa_mocore_%s_FY_raw' % str(ii)])

resa_mocore_27_FY.head()

,sym,lvl,7,11,12,13,14,17,18,20,21
num-1,,,,,,,,,,,
1,C,s,0.0,99.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,C,s,0.0,0.0,0.0,0.3,0.3,0.0,0.0,3.6,95.4
11,C,s,0.0,0.0,0.0,0.3,98.7,0.0,0.0,0.2,0.5
2,C,s,99.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,C,s,0.0,0.0,0.0,0.0,0.0,0.2,99.3,0.0,0.0


<!DOCTYPE html>
<html>

<head>
<meta charset="UTF-8">
<title>Title of the document</title>
</head>

<body>
<h3 id="title">Reducing Löwdin Population MO matrices to resB boundaries</h3>
</body>
</html>

In [43]:
for ii in list_suf:
    globals()['resb_movirt_%s_FY' % str(ii)] = nonzero_mo_matrix(globals()['resb_movirt_%s_FY_raw' % str(ii)])

resb_movirt_27_FY.head()

,sym,lvl,92,93,94,95,96,97,98,99,100,101,102,103
num-1,,,,,,,,,,,,,,
23,atom,lvlMO,0.0,0.0,0.0,0.0,0.9,2.9,0.6,1.2,3.2,0.6,1.3,1.2
24,atom,lvlMO,0.4,0.8,0.0,0.0,2.2,3.8,1.5,1.7,3.8,1.1,0.5,1.9
25,atom,lvlMO,0.2,0.0,0.0,0.0,1.2,22.2,7.7,12.8,1.3,2.1,3.0,3.5
26,atom,lvlMO,0.0,0.0,0.0,0.0,0.3,10.1,3.7,5.8,0.5,0.8,1.8,1.4
27,atom,lvlMO,0.0,0.0,0.0,0.0,0.4,7.5,2.7,7.7,0.5,0.5,0.8,0.8


<!DOCTYPE html>
<html>

<head>
<meta charset="UTF-8">
<title>Title of the document</title>
</head>

<body>
<h3 id="title">Reducing further resB Löwdin Population MO just considering:</h3>
<h4 id="title">67% as population limit</h4>
</body>
</html>

In [48]:
#for counting transitions in MO having greater than 67% of Loewdin population contribution in resB
pop = 0.67
for ii in list_suf:
    globals()['resb_movirt_%s_FY_pop' % str(ii)] = crop_by_loewdin_p(globals()['resb_movirt_%s_FY_raw' % str(ii)],pop)

resb_movirt_27_FY_pop.head()

,sym,lvl,94,97,99,100
num-1,,,,,,
23,atom,lvlMO,0.0,2.9,1.2,3.2
24,atom,lvlMO,0.0,3.8,1.7,3.8
25,atom,lvlMO,0.0,22.2,12.8,1.3
26,atom,lvlMO,0.0,10.1,5.8,0.5
27,atom,lvlMO,0.0,7.5,7.7,0.5


In [18]:
#[int(i) for i in resb_movirt_G5FG5Y.axes[1][2:].tolist()]

In [55]:
list_suf = [27]
for ii in list_suf:
    globals()['core_virt_mo_%s_FY' % str(ii)] = cropping_matrix(globals()['corevirtMO_matrix_%s_FY_raw' % str(ii)], globals()['resa_mocore_%s_FY' % str(ii)], globals()['resb_movirt_%s_FY' % str(ii)])

core_virt_mo_27_FY.head()

,7,11,12,13,14,17,18,20,21
virt\core,,,,,,,,,
92,3,1,1,4,3,2,1,2,5
93,5,2,1,5,4,4,1,4,4
94,5,2,1,6,5,4,2,4,4
95,5,1,1,5,5,4,3,4,4
96,5,1,1,4,4,4,2,4,5


In [56]:
list_suf = [27]
for ii in list_suf:
    globals()['core_virt_mo_%s_FY_pop' % str(ii)] = cropping_matrix(globals()['corevirtMO_matrix_%s_FY_raw' % str(ii)], globals()['resa_mocore_%s_FY' % str(ii)], globals()['resb_movirt_%s_FY_pop' % str(ii)])

core_virt_mo_27_FY_pop.head()

,7,11,12,13,14,17,18,20,21
virt\core,,,,,,,,,
94,5,2,1,6,5,4,2,4,4
97,4,1,1,3,4,3,1,3,4
99,6,2,2,2,2,4,2,3,3
100,7,2,3,2,3,5,2,3,3


In [20]:
core_virt_mo_tspb_G9FGY = cropping_matrix(core_virt_mo_tspb_G9FGY_raw, resa_mocore_G9FGY, resb_movirt_G9FGY)
core_virt_mo_tspb_G7FG3Y = cropping_matrix(core_virt_mo_tspb_G7FG3Y_raw, resa_mocore_G7FG3Y, resb_movirt_G7FG3Y)
core_virt_mo_tspb_G5FG5Y = cropping_matrix(core_virt_mo_tspb_G5FG5Y_raw, resa_mocore_G5FG5Y, resb_movirt_G5FG5Y)
core_virt_mo_tspb_G3FG7Y = cropping_matrix(core_virt_mo_tspb_G3FG7Y_raw, resa_mocore_G3FG7Y, resb_movirt_G3FG7Y)
core_virt_mo_tspb_GFG9Y = cropping_matrix(core_virt_mo_tspb_GFG9Y_raw, resa_mocore_GFG9Y, resb_movirt_GFG9Y)
#core_virt_mo_GFG9Y

core_virt_mo_tspb_G9FGY_pop = cropping_matrix(core_virt_mo_tspb_G9FGY_raw, resa_mocore_G9FGY, resb_movirt_G9FGY_pop)
core_virt_mo_tspb_G7FG3Y_pop = cropping_matrix(core_virt_mo_tspb_G7FG3Y_raw, resa_mocore_G7FG3Y, resb_movirt_G7FG3Y_pop)
core_virt_mo_tspb_G5FG5Y_pop = cropping_matrix(core_virt_mo_tspb_G5FG5Y_raw, resa_mocore_G5FG5Y, resb_movirt_G5FG5Y_pop)
core_virt_mo_tspb_G3FG7Y_pop = cropping_matrix(core_virt_mo_tspb_G3FG7Y_raw, resa_mocore_G3FG7Y, resb_movirt_G3FG7Y_pop)
core_virt_mo_tspb_GFG9Y_pop = cropping_matrix(core_virt_mo_tspb_GFG9Y_raw, resa_mocore_GFG9Y, resb_movirt_GFG9Y_pop)

In [21]:
core_virt_mo_fosce_G9FGY = cropping_matrix(core_virt_mo_fosce_G9FGY_raw, resa_mocore_G9FGY, resb_movirt_G9FGY)
core_virt_mo_fosce_G7FG3Y = cropping_matrix(core_virt_mo_fosce_G7FG3Y_raw, resa_mocore_G7FG3Y, resb_movirt_G7FG3Y)
core_virt_mo_fosce_G5FG5Y = cropping_matrix(core_virt_mo_fosce_G5FG5Y_raw, resa_mocore_G5FG5Y, resb_movirt_G5FG5Y)
core_virt_mo_fosce_G3FG7Y = cropping_matrix(core_virt_mo_fosce_G3FG7Y_raw, resa_mocore_G3FG7Y, resb_movirt_G3FG7Y)
core_virt_mo_fosce_GFG9Y = cropping_matrix(core_virt_mo_fosce_GFG9Y_raw, resa_mocore_GFG9Y, resb_movirt_GFG9Y)

core_virt_mo_fosce_G9FGY_pop = cropping_matrix(core_virt_mo_fosce_G9FGY_raw, resa_mocore_G9FGY, resb_movirt_G9FGY_pop)
core_virt_mo_fosce_G7FG3Y_pop = cropping_matrix(core_virt_mo_fosce_G7FG3Y_raw, resa_mocore_G7FG3Y, resb_movirt_G7FG3Y_pop)
core_virt_mo_fosce_G5FG5Y_pop = cropping_matrix(core_virt_mo_fosce_G5FG5Y_raw, resa_mocore_G5FG5Y, resb_movirt_G5FG5Y_pop)
core_virt_mo_fosce_G3FG7Y_pop = cropping_matrix(core_virt_mo_fosce_G3FG7Y_raw, resa_mocore_G3FG7Y, resb_movirt_G3FG7Y_pop)
core_virt_mo_fosce_GFG9Y_pop = cropping_matrix(core_virt_mo_fosce_GFG9Y_raw, resa_mocore_GFG9Y, resb_movirt_GFG9Y_pop)
#core_virt_mo_fosce_G9FGY.head()

core_virt_mo_foscv_G9FGY = cropping_matrix(core_virt_mo_foscv_G9FGY_raw, resa_mocore_G9FGY, resb_movirt_G9FGY)
core_virt_mo_foscv_G7FG3Y = cropping_matrix(core_virt_mo_foscv_G7FG3Y_raw, resa_mocore_G7FG3Y, resb_movirt_G7FG3Y)
core_virt_mo_foscv_G5FG5Y = cropping_matrix(core_virt_mo_foscv_G5FG5Y_raw, resa_mocore_G5FG5Y, resb_movirt_G5FG5Y)
core_virt_mo_foscv_G3FG7Y = cropping_matrix(core_virt_mo_foscv_G3FG7Y_raw, resa_mocore_G3FG7Y, resb_movirt_G3FG7Y)
core_virt_mo_foscv_GFG9Y = cropping_matrix(core_virt_mo_foscv_GFG9Y_raw, resa_mocore_GFG9Y, resb_movirt_GFG9Y)

core_virt_mo_foscv_G9FGY_pop = cropping_matrix(core_virt_mo_foscv_G9FGY_raw, resa_mocore_G9FGY, resb_movirt_G9FGY_pop)
core_virt_mo_foscv_G7FG3Y_pop = cropping_matrix(core_virt_mo_foscv_G7FG3Y_raw, resa_mocore_G7FG3Y, resb_movirt_G7FG3Y_pop)
core_virt_mo_foscv_G5FG5Y_pop = cropping_matrix(core_virt_mo_foscv_G5FG5Y_raw, resa_mocore_G5FG5Y, resb_movirt_G5FG5Y_pop)
core_virt_mo_foscv_G3FG7Y_pop = cropping_matrix(core_virt_mo_foscv_G3FG7Y_raw, resa_mocore_G3FG7Y, resb_movirt_G3FG7Y_pop)
core_virt_mo_foscv_GFG9Y_pop = cropping_matrix(core_virt_mo_foscv_GFG9Y_raw, resa_mocore_GFG9Y, resb_movirt_GFG9Y_pop)
#core_virt_mo_foscv_G9FGY_pop.head()

In [22]:
#core_virt_mo_tspb_G3FG7Y_pop.head()

In [23]:
core_virt_mo_G9FGY.shape
#core_virt_mo_tspb_G9FGY.shape
#core_virt_mo_G9FGY

(21, 8)

In [24]:
core_virt_mo_G9FGY_pop.shape

(3, 8)

In [25]:
core_virt_mo_G7FG3Y.shape
#core_virt_mo_tspb_G7FG3Y.shape

(19, 9)

In [26]:
core_virt_mo_G5FG5Y.shape
#core_virt_mo_tspb_G5FG5Y.shape

(16, 9)

In [27]:
core_virt_mo_G3FG7Y.shape
#core_virt_mo_tspb_G3FG7Y.shape

(16, 9)

In [28]:
core_virt_mo_GFG9Y.shape
#core_virt_mo_tspb_GFG9Y.shape

(20, 9)

In [29]:
core_virt_mo_G9FGY_raw.sum().sum()

1559

In [30]:
core_virt_mo_G9FGY_raw.shape

(26, 35)

In [31]:
core_virt_mo_G7FG3Y_raw.sum().sum()

1175

In [32]:
core_virt_mo_G5FG5Y_raw.sum().sum()

1206

In [33]:
core_virt_mo_G3FG7Y_raw.sum().sum()

1218

In [34]:
core_virt_mo_GFG9Y_raw.sum().sum()

1423

Comparisson

In [35]:
core_virt_mo_G9FGY.sum().sum()/core_virt_mo_G9FGY_raw.sum().sum()

0.5522771007055804

In [36]:
core_virt_mo_G7FG3Y.sum().sum()/core_virt_mo_G7FG3Y_raw.sum().sum()

0.5880851063829787

In [37]:
core_virt_mo_G5FG5Y.sum().sum()/core_virt_mo_G5FG5Y_raw.sum().sum()

0.48175787728026537

In [38]:
core_virt_mo_G3FG7Y.sum().sum()/core_virt_mo_G3FG7Y_raw.sum().sum()

0.513136288998358

In [39]:
core_virt_mo_GFG9Y.sum().sum()/core_virt_mo_GFG9Y_raw.sum().sum()

0.4483485593815882

#### 50% loewdin population

In [40]:
core_virt_mo_G9FGY_pop.sum().sum(), core_virt_mo_G9FGY.sum().sum()

(152, 861)

In [41]:
core_virt_mo_G7FG3Y_pop.sum().sum(), core_virt_mo_G7FG3Y.sum().sum()

(137, 691)

In [42]:
core_virt_mo_G5FG5Y_pop.sum().sum(), core_virt_mo_G5FG5Y.sum().sum()

(98, 581)

In [43]:
core_virt_mo_G3FG7Y_pop.sum().sum(), core_virt_mo_G3FG7Y.sum().sum()

(86, 625)

In [44]:
core_virt_mo_GFG9Y_pop.sum().sum(), core_virt_mo_GFG9Y.sum().sum()

(9, 638)

#### 67% loewdin population - weighted fosc electronic dipole moment

In [45]:
core_virt_mo_fosce_G9FGY_pop.sum().sum(), core_virt_mo_fosce_G9FGY.sum().sum()

(0.03383399445, 0.091651643142)

In [46]:
core_virt_mo_fosce_G7FG3Y_pop.sum().sum(), core_virt_mo_fosce_G7FG3Y.sum().sum()

(0.04438726753000001, 0.1691764115875)

In [47]:
core_virt_mo_fosce_G5FG5Y_pop.sum().sum(), core_virt_mo_fosce_G5FG5Y.sum().sum()

(0.006232323400000001, 0.1245451627907)

In [48]:
core_virt_mo_fosce_G3FG7Y_pop.sum().sum(), core_virt_mo_fosce_G3FG7Y.sum().sum()

(0.003689181141, 0.1573194259684)

In [49]:
core_virt_mo_fosce_GFG9Y_pop.sum().sum(), core_virt_mo_fosce_GFG9Y.sum().sum()

(4.328911399999999e-06, 0.17220337080499998)

In [50]:
core_virt_mo_G9FGY_pop

,40,52,53,57,58,60,61,62
virt\core,,,,,,,,
237,0,3,3,4,6,6,5,6
240,1,6,5,6,6,11,4,8
246,0,12,11,9,9,11,10,10


In [51]:
core_virt_mo_G3FG7Y_pop

,35,44,53,55,59,60,61,62,63
virt\core,,,,,,,,,
242,0,0,1,0,6,5,3,2,6
247,1,1,9,10,13,8,7,8,6


In [52]:
core_virt_mo_fosce_G9FGY_pop

,40,52,53,57,58,60,61,62
virt\core,,,,,,,,
237,0.000000,0.000064,0.000025,0.000023,0.000003,0.000002,0.000249,0.000024
240,0.006429,0.004824,0.002107,0.004528,0.002711,0.004084,0.008399,0.000216
246,0.000000,0.000011,0.000046,0.000002,0.000012,0.000033,0.000017,0.000023


In [53]:
core_virt_mo_fosce_G3FG7Y_pop

,35,44,53,55,59,60,61,62,63
virt\core,,,,,,,,,
242,0.000000,0.000000,0.000012,0.00000,8.608450e-05,0.000165,0.000377,2.556440e-05,0.001109
247,0.001397,0.000436,0.000015,0.00001,2.859550e-07,0.000003,0.000008,6.685260e-07,0.000046


In [54]:
core_virt_mo_foscv_G9FGY_pop

,40,52,53,57,58,60,61,62
virt\core,,,,,,,,
237,0.00000,0.000061,0.000024,0.000022,0.000004,0.000002,0.000244,0.000024
240,0.00608,0.004504,0.001965,0.004232,0.002523,0.003801,0.007829,0.000202
246,0.00000,0.000011,0.000045,0.000002,0.000011,0.000032,0.000016,0.000022


In [55]:
core_virt_mo_tspb_G9FGY_pop.sum().sum()/core_virt_mo_tspb_G9FGY_pop.count().sum()#.sum().sum()#/core_virt_mo_tspb_G9FGY

0.09561253041666667

In [56]:
core_virt_mo_tspb_G7FG3Y_pop.sum().sum()/core_virt_mo_tspb_G7FG3Y_pop.count().sum()#.sum().sum()

0.09464789148148148

In [57]:
core_virt_mo_tspb_G5FG5Y_pop.sum().sum()/core_virt_mo_tspb_G5FG5Y_pop.count().sum()#.sum().sum()

0.009850825185185185

In [58]:
core_virt_mo_tspb_G3FG7Y_pop.sum().sum()/core_virt_mo_tspb_G3FG7Y_pop.count().sum()#.sum().sum()

0.023667219444444443

In [59]:
core_virt_mo_tspb_G5FG5Y_pop.sum().sum()/core_virt_mo_tspb_GFG9Y_pop.count().sum()#.sum().sum()

0.029552475555555555

In [60]:
core_virt_mo_G7FG3Y_pop

,31,40,53,55,59,60,61,62,63
virt\core,,,,,,,,,
237,0,0,5,6,14,8,7,12,15
238,0,2,4,3,2,5,4,4,4
241,0,3,4,2,4,8,8,9,4


In [61]:
core_virt_mo_fosce_G9FGY_pop.sum().sum()/core_virt_mo_fosce_G9FGY_pop.count().sum()

0.0014097497687500001

In [62]:
core_virt_mo_fosce_G7FG3Y_pop.sum().sum()/core_virt_mo_fosce_G7FG3Y_pop.count().sum()

0.0016439728714814819

In [63]:
core_virt_mo_fosce_G5FG5Y_pop.sum().sum()/core_virt_mo_fosce_G5FG5Y_pop.count().sum()

0.0002308267925925926

In [64]:
core_virt_mo_fosce_G3FG7Y_pop.sum().sum()/core_virt_mo_fosce_G3FG7Y_pop.count().sum()

0.00020495450783333333

In [65]:
core_virt_mo_fosce_GFG9Y_pop.sum().sum()/core_virt_mo_fosce_GFG9Y_pop.count().sum()

4.809901555555555e-07

In [66]:
core_virt_mo_tspb_G9FGY.sum().sum()#/core_virt_mo_G5FG5Y.sum().sum()
#core_virt_mo_tspb_G9FGY.sum().sum()/core_virt_mo_tspb_G9FGY_raw.sum().sum()

4.2172563400000005

In [67]:
core_virt_mo_tspb_G7FG3Y.sum().sum()#/core_virt_mo_G7FG3Y.sum().sum()
#core_virt_mo_tspb_G7FG3Y.sum().sum()/core_virt_mo_tspb_G7FG3Y_raw.sum().sum()

15.785869139999999

In [68]:
core_virt_mo_tspb_G5FG5Y.sum().sum()#/core_virt_mo_G5FG5Y.sum().sum()
#core_virt_mo_tspb_G5FG5Y.sum().sum()/core_virt_mo_tspb_G5FG5Y_raw.sum().sum()

8.28772924

In [69]:
core_virt_mo_tspb_G3FG7Y.sum().sum()#/core_virt_mo_G3FG7Y.sum().sum()
#core_virt_mo_tspb_G3FG7Y.sum().sum()/core_virt_mo_tspb_G3FG7Y_raw.sum().sum()

10.33902286

In [70]:
core_virt_mo_tspb_GFG9Y.sum().sum()#/core_virt_mo_GFG9Y.sum().sum()
#core_virt_mo_tspb_GFG9Y.sum().sum()/core_virt_mo_tspb_GFG9Y_raw.sum().sum()

26.060408999999996

In [71]:
core_virt_mo_GFG9Y.sum().sum()

638

In [72]:
core_virt_mo_fosce_G9FGY.sum().sum()

0.091651643142

In [73]:
core_virt_mo_fosce_G7FG3Y.sum().sum()

0.1691764115875

In [74]:
core_virt_mo_fosce_G5FG5Y.sum().sum()

0.1245451627907

In [75]:
core_virt_mo_fosce_G3FG7Y.sum().sum()

0.1573194259684

In [76]:
core_virt_mo_fosce_GFG9Y.sum().sum()

0.17220337080499998

In [77]:
core_virt_mo_G9FGY

,40,52,53,57,58,60,61,62
virt\core,,,,,,,,
237,0,3,3,4,6,6,5,6
238,1,8,8,10,6,4,9,11
239,1,8,7,8,8,4,10,9
240,1,6,5,6,6,11,4,8
241,1,4,5,12,6,7,8,7
242,0,7,7,14,11,9,7,8
243,0,12,10,14,11,9,9,8
244,0,0,0,13,1,1,7,9
245,0,8,7,9,9,7,12,7


In [78]:
resb_movirt_G9FGY

,sym,lvl,237,238,239,240,241,242,243,244,...,248,249,250,251,252,253,256,259,260,262
num-1,,,,,,,,,,,,,,,,,,,,,
91,atom,lvlMO,0.0,0.1,0.2,1.2,0.0,1.9,0.3,0.2,...,0.9,0.6,0.7,1.4,0.9,1.8,0.0,0.1,2.2,1.7
92,atom,lvlMO,0.4,0.1,0.5,2.2,0.0,3.5,0.4,0.1,...,0.1,0.1,0.5,0.4,0.2,0.3,0.0,0.0,0.4,0.3
93,atom,lvlMO,2.2,0.4,0.6,5.1,0.0,18.9,3.7,1.2,...,0.3,1.7,0.7,2.6,0.7,1.5,0.0,0.2,4.8,0.5
94,atom,lvlMO,1.2,0.2,0.3,2.6,0.0,9.0,1.8,0.5,...,0.2,0.8,0.4,1.2,0.4,0.8,0.0,0.0,1.4,0.2
95,atom,lvlMO,0.5,0.1,0.1,0.9,0.0,3.5,0.7,0.2,...,0.6,1.1,1.9,1.7,0.8,1.4,0.0,0.0,0.6,0.2
96,atom,lvlMO,0.5,0.5,0.6,2.5,0.0,3.1,0.6,0.1,...,0.6,0.2,0.8,0.2,0.2,0.2,0.0,0.0,1.8,1.0
97,atom,lvlMO,1.0,0.8,0.2,15.2,1.6,5.9,0.0,0.2,...,0.1,0.5,2.0,0.4,0.0,0.1,0.0,0.0,0.5,0.1
98,atom,lvlMO,13.4,1.0,3.0,7.7,0.9,5.6,0.1,0.0,...,0.0,0.0,0.6,0.1,0.0,0.0,0.0,0.0,1.0,0.3
99,atom,lvlMO,18.8,4.1,1.8,3.6,0.2,0.7,0.1,0.1,...,0.5,0.3,1.8,0.1,0.1,0.2,0.0,0.0,2.2,0.1


In [79]:
resb_movirt_GFG9Y[:].sum()[2:]
((resb_movirt_GFG9Y.sum()[2:]>50)==True).index#.tolist()
#((resb_movirt_GFG9Y[:].sum()[2:]>50)==True).count()
#resb_movirt_GFG9Y.apply(tuple, axis=1)

Index(['237', '238', '240', '241', '242', '243', '244', '245', '246', '247',
       '248', '249', '250', '251', '252', '253', '255', '256', '257', '260'],
      dtype='object')

In [80]:
resb_movirt_GFG9Y.sum()[2:].index#[0]

Index(['237', '238', '240', '241', '242', '243', '244', '245', '246', '247',
       '248', '249', '250', '251', '252', '253', '255', '256', '257', '260'],
      dtype='object')

In [81]:
resb_movirt_GFG9Y.sum()[2:].loc[resb_movirt_GFG9Y.sum()[2:].index]

237    98.2
238    46.2
240    20.3
241    53.1
242    28.0
243    12.1
244     9.6
245    57.0
246    28.8
247    14.7
248    28.4
249    31.3
250    46.7
251     0.3
252    59.5
253     0.1
255     6.1
256     4.6
257     0.3
260     4.1
dtype: object

In [82]:
#resb_movirt_GFG9Y
crop_by_loewdin_p(resb_movirt_GFG9Y,0.5)

,sym,lvl,237,241,245,252
num-1,,,,,,
91,atom,lvlMO,0.0,1.3,3.0,3.2
92,atom,lvlMO,0.6,2.5,1.1,0.4
93,atom,lvlMO,2.3,12.4,6.2,0.6
94,atom,lvlMO,1.2,5.7,2.4,0.8
95,atom,lvlMO,0.5,2.3,5.1,2.4
96,atom,lvlMO,0.5,1.9,1.0,1.2
97,atom,lvlMO,1.5,6.1,0.6,0.9
98,atom,lvlMO,16.0,5.8,0.8,0.1
99,atom,lvlMO,24.9,0.5,0.1,0.9


In [83]:
core_virt_mo_GFG9Y

,36,48,56,58,59,60,61,62,63
virt\core,,,,,,,,,
237,0,0,1,1,1,1,1,2,2
238,0,0,0,0,2,1,1,2,2
240,0,0,1,1,1,1,2,3,2
241,1,1,3,2,3,5,4,8,2
242,0,0,1,2,1,2,1,3,3
243,1,0,3,5,4,7,8,4,6
244,0,0,1,1,3,2,3,1,6
245,0,0,3,5,4,5,7,7,4
246,0,0,5,5,6,7,8,7,5


##### dx,dy and dz correspond to the x-, y- and z-components of the transition moment (d)

### G$_{2}$YG$_{n}$PG$_ {11-n}$ ( $n=0..9$ )
#### optimized at PM7 and DFTB3-D3H4
#### by similarity

In [84]:
#Uncomment the original range to see more peaks outside the expected range
file=""
post="_opt_gms.csv"
g2ypg11 = pd.read_csv(file+'G2YPG11'+post, delimiter=',')
g2ygpg10 = pd.read_csv(file+'G2YGPG10'+post, delimiter=',')
g2yg2pg9 = pd.read_csv(file+'G2YG2PG9'+post, delimiter=',')
g2yg3pg8 = pd.read_csv(file+'G2YG3PG8'+post, delimiter=',')
g2yg4pg7 = pd.read_csv(file+'G2YG4PG7'+post, delimiter=',')
g2yg5pg6 = pd.read_csv(file+'G2YG5PG6'+post, delimiter=',')
g2yg6pg5 = pd.read_csv(file+'G2YG6PG5'+post, delimiter=',')
g2yg7pg4 = pd.read_csv(file+'G2YG7PG4'+post, delimiter=',')
g2yg8pg3 = pd.read_csv(file+'G2YG8PG3'+post, delimiter=',')
g2yg9pg2 = pd.read_csv(file+'G2YG9PG2'+post, delimiter=',')
g2yg12 = pd.read_csv(file+'G2YG12'+post, delimiter=',')
g4pg10 = pd.read_csv(file+'G4PG10'+post, delimiter=',')

g2ypg11.columns = ['Energy', 'Intensity', 'dx', 'dy', 'dz']
g2ygpg10.columns = ['Energy', 'Intensity', 'dx', 'dy', 'dz']
g2yg2pg9.columns = ['Energy', 'Intensity', 'dx', 'dy', 'dz']
g2yg3pg8.columns = ['Energy', 'Intensity', 'dx', 'dy', 'dz']
g2yg4pg7.columns = ['Energy', 'Intensity', 'dx', 'dy', 'dz']
g2yg5pg6.columns = ['Energy', 'Intensity', 'dx', 'dy', 'dz']
g2yg6pg5.columns = ['Energy', 'Intensity', 'dx', 'dy', 'dz']
g2yg7pg4.columns = ['Energy', 'Intensity', 'dx', 'dy', 'dz']
g2yg8pg3.columns = ['Energy', 'Intensity', 'dx', 'dy', 'dz']
g2yg9pg2.columns = ['Energy', 'Intensity', 'dx', 'dy', 'dz']
g2yg12.columns = ['Energy', 'Intensity', 'dx', 'dy', 'dz']
g4pg10.columns = ['Energy', 'Intensity', 'dx', 'dy', 'dz']

g2yg9pg2

FileNotFoundError: [Errno 2] No such file or directory: 'G2YPG11_opt_gms.csv'

In [ ]:
#add yft and yfc to see a broader energy range
fig = plt.figure(figsize=(14, 10), dpi=80)
n2 = fig.add_subplot(111)
#n2.scatter(g2ypg11[['Energy']],g2ypg11[['Intensity']], s=5, label='G2YPG11')
#n2.scatter(g2ygpg10[['Energy']],g2ygpg10[['Intensity']], s=5, label='G2YGPG10')
#n2.scatter(g2yg2pg9[['Energy']],g2yg2pg9[['Intensity']], s=5, label='G2YG2PG9')
#n2.scatter(g2yg3pg8[['Energy']],g2yg3pg8[['Intensity']], s=5, label='G2YG3PG8')
#n2.scatter(g2yg4pg7[['Energy']],g2yg4pg7[['Intensity']], s=5, label='G2YG4PG7')
#n2.scatter(g2yg5pg6[['Energy']],g2yg5pg6[['Intensity']], s=5, label='G2YG5PG6')
n2.scatter(g2yg6pg5[['Energy']],g2yg6pg5[['Intensity']], s=5, label='G2YG6PG5')
n2.scatter(g2yg7pg4[['Energy']],g2yg7pg4[['Intensity']], s=5, label='G2YG7PG4')
n2.scatter(g2yg8pg3[['Energy']],g2yg8pg3[['Intensity']], s=5, label='G2YG8PG3')
n2.scatter(g2yg9pg2[['Energy']],g2yg9pg2[['Intensity']], s=5, label='G2YG9PG2')
plt.xlabel('Energy (eV)')
plt.ylabel('Intensity (Oscillator strength)')
plt.legend(loc='upper left');
plt.show()

In [ ]:
#add yft and yfc to see a broader energy range
fig = plt.figure(figsize=(14, 10), dpi=80)
n2 = fig.add_subplot(111)
n2.scatter(g2ypg11[['Energy']],g2ypg11[['Intensity']], s=5, label='G2YPG11')
n2.scatter(g2ygpg10[['Energy']],g2ygpg10[['Intensity']], s=5, label='G2YGPG10')
n2.scatter(g2yg2pg9[['Energy']],g2yg2pg9[['Intensity']], s=5, label='G2YG2PG9')
n2.scatter(g2yg3pg8[['Energy']],g2yg3pg8[['Intensity']], s=5, label='G2YG3PG8')
n2.scatter(g2yg4pg7[['Energy']],g2yg4pg7[['Intensity']], s=5, label='G2YG4PG7')
#n2.scatter(g2yg5pg6[['Energy']],g2yg5pg6[['Intensity']], s=5, label='G2YG5PG6')
#n2.scatter(g2yg6pg5[['Energy']],g2yg6pg5[['Intensity']], s=5, label='G2YG6PG5')
#n2.scatter(g2yg7pg4[['Energy']],g2yg7pg4[['Intensity']], s=5, label='G2YG7PG4')
#n2.scatter(g2yg8pg3[['Energy']],g2yg8pg3[['Intensity']], s=5, label='G2YG8PG3')
#n2.scatter(g2yg9pg2[['Energy']],g2yg9pg2[['Intensity']], s=5, label='G2YG9PG2')
plt.xlabel('Energy (eV)')
plt.ylabel('Intensity (Oscillator strength)')
plt.legend(loc='upper left');
plt.show()

In [ ]:
#add yft and yfc to see a broader energy range
fig = plt.figure(figsize=(14, 10), dpi=80)
n2 = fig.add_subplot(111)
#n2.scatter(g2ypg11[['Energy']],g2ypg11[['Intensity']], s=5, label='G2YPG11')
#n2.scatter(g2ygpg10[['Energy']],g2ygpg10[['Intensity']], s=5, label='G2YGPG10')
#n2.scatter(g2yg2pg9[['Energy']],g2yg2pg9[['Intensity']], s=5, label='G2YG2PG9')
#n2.scatter(g2yg3pg8[['Energy']],g2yg3pg8[['Intensity']], s=5, label='G2YG3PG8')
n2.scatter(g2yg4pg7[['Energy']],g2yg4pg7[['Intensity']], s=5, label='G2YG4PG7')
n2.scatter(g2yg5pg6[['Energy']],g2yg5pg6[['Intensity']], s=5, label='G2YG5PG6')
n2.scatter(g2yg6pg5[['Energy']],g2yg6pg5[['Intensity']], s=5, label='G2YG6PG5')
#n2.scatter(g2yg7pg4[['Energy']],g2yg7pg4[['Intensity']], s=5, label='G2YG7PG4')
#n2.scatter(g2yg8pg3[['Energy']],g2yg8pg3[['Intensity']], s=5, label='G2YG8PG3')
#n2.scatter(g2yg9pg2[['Energy']],g2yg9pg2[['Intensity']], s=5, label='G2YG9PG2')
plt.xlabel('Energy (eV)')
plt.ylabel('Intensity (Oscillator strength)')
plt.legend(loc='upper left');
plt.show()

In [ ]:
#add yft and yfc to see a broader energy range
fig = plt.figure(figsize=(14, 10), dpi=80)
n2 = fig.add_subplot(111)
#n2.scatter(g2ypg11[['Energy']],g2ypg11[['Intensity']], s=5, label='G2YPG11')
#n2.scatter(g2ygpg10[['Energy']]-0.05,g2ygpg10[['Intensity']], s=5, label='G2YGPG10')
#n2.scatter(g2yg2pg9[['Energy']]-0.13,g2yg2pg9[['Intensity']], s=5, label='G2YG2PG9')
#n2.scatter(g2yg3pg8[['Energy']]-0.21,g2yg3pg8[['Intensity']], s=5, label='G2YG3PG8')
n2.scatter(g2yg4pg7[['Energy']],g2yg4pg7[['Intensity']], s=5, label='G2YG4PG7')
n2.scatter(g2yg5pg6[['Energy']],g2yg5pg6[['Intensity']], s=5, label='G2YG5PG6')
n2.scatter(g2yg6pg5[['Energy']],g2yg6pg5[['Intensity']], s=5, label='G2YG6PG5')
n2.scatter(g2yg7pg4[['Energy']],g2yg7pg4[['Intensity']], s=5, label='G2YG7PG4')
#n2.scatter(g2yg8pg3[['Energy']]-0.4,g2yg8pg3[['Intensity']], s=5, label='G2YG8PG3')
#n2.scatter(g2yg9pg2[['Energy']]-0.25,g2yg9pg2[['Intensity']], s=5, label='G2YG9PG2')
n2.scatter(g2yg12[['Energy']],g2yg12[['Intensity']], s=5, label='G2YG12')
n2.scatter(g4pg10[['Energy']],g4pg10[['Intensity']], s=5, label='G4PG10')
plt.xlabel('Energy (eV)')
plt.ylabel('Intensity (Oscillator strength)')
plt.legend(loc='upper left');
plt.show()